In [5]:
import pandas as pd
import re

# Punto 1 

Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.


In [3]:
# importiamo il dataset
loans_len=pd.read_csv('loans_lenders.csv')

In [4]:
loans_len.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [5]:
#Selezioniamo solamente l'Id del loan
loan_id=loans_len['loan_id']
loan_id

0           483693
1           483738
2           485000
3           486087
4           534428
            ...   
1387427     678999
1387428    1207353
1387429    1206220
1387430    1206425
1387431    1206486
Name: loan_id, Length: 1387432, dtype: int64

In [6]:
# con la funzione explode abbiamo separato le stringhe dei lenders ogni volta è presente la virgola attribuendole allo stesso indice
sing_lend=loans_len['lenders'].str.split(',').explode()
sing_lend

0                      muc888
0                     sam4326
0                 camaran3922
0                 lachheb1865
0                 rebecca3499
                  ...        
1387430     trogdorfamily7622
1387430             danny6470
1387430               don6118
1387431              alan5175
1387431           amy38101311
Name: lenders, Length: 28293931, dtype: object

In [7]:
# la funzione strip è servita per eliminare gli spazi bianchi tra i lender
sing_lend=sing_lend.str.strip()

In [8]:
# abbiamo creato il dataset assegnando ad ogni lender il suo loan iniziale
loan_id=pd.Series(loans_len['loan_id'])

ab={ 'loan_id': loan_id , 'lenders': sing_lend}
fin_fin=pd.DataFrame(ab)

In [9]:
fin_fin

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499
...,...,...
1387430,1206425,trogdorfamily7622
1387430,1206425,danny6470
1387430,1206425,don6118
1387431,1206486,alan5175


In [10]:
fin_fin = fin_fin.reset_index(drop=True)
fin_fin

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


# Punto 2


For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.


In [11]:
import pandas as pd
loans=pd.read_csv('loans.csv')

In [12]:
import datetime
import numpy as np

In [13]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner


In [14]:
# notiamo che molte date presentano un disburse time maggiore dell'expiration time 
# crediamo che questo sia un errore in quanto la data scadenza del prestito è precedente a quella di esborso
loans.loc[loans['disburse_time'] > loans['planned_expiration_time']]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model
63,1158981,Rabiu,English,Rabiu is a Babban Gona maize farmer from Kwany...,Rabiu is a Babban Gona maize farmer from Kwany...,250.0,250.0,funded,Personal Expenses,Personal Use,...,2016-10-31 05:10:53.000 +0000,15.0,10,1,1,"#Vegan, #Repeat Borrower, #Supporting Family",male,true,bullet,field_partner
96,961161,Usman,English,Usman is a Babban Gona maize farmer. Thanks to...,Usman is a Babban Gona maize farmer. Thanks to...,200.0,200.0,funded,Personal Expenses,Personal Use,...,2015-11-12 00:39:38.000 +0000,14.0,8,2,2,#Vegan,male,true,bullet,field_partner
177,961701,Aminu,English,Aminu is a maize farmer and Babban Gona member...,Aminu is a maize farmer and Babban Gona member...,100.0,100.0,funded,Personal Expenses,Personal Use,...,2015-10-17 18:00:34.000 +0000,14.0,3,2,2,"#Parent, #Vegan",male,true,bullet,field_partner
207,1155776,Salisu,English,Salisu is a maize farmer from U/Dashire villag...,Salisu is a maize farmer from U/Dashire villag...,200.0,200.0,funded,Primary/secondary school costs,Education,...,2016-10-04 21:56:50.000 +0000,15.0,8,1,1,"#Schooling, #Vegan, #Repeat Borrower, #Parent",male,true,bullet,field_partner
217,1158968,Musa,English,Musa is a Babban Gona maize farmer from the vi...,Musa is a Babban Gona maize farmer from the vi...,125.0,125.0,funded,Personal Expenses,Personal Use,...,2016-10-03 20:56:13.000 +0000,15.0,5,1,1,"#Vegan, #Repeat Borrower",male,true,bullet,field_partner
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419303,1067265,Juwyana,English,Juwayana is a Babban Gona maize farmer. She is...,Juwayana is a Babban Gona maize farmer. She is...,100.0,100.0,funded,Farming,Agriculture,...,2016-05-13 16:13:13.000 +0000,19.0,3,1,1,"#Parent, #Schooling, #Vegan",female,true,bullet,field_partner
1419304,1067985,Aliyu,English,Aliyu is a Babban Gona maize farmer from Turaw...,Aliyu is a Babban Gona maize farmer from Turaw...,350.0,350.0,funded,Farming,Agriculture,...,2016-06-11 13:46:05.000 +0000,19.0,14,1,1,"#Parent, #Schooling, #Vegan, #Sustainable Ag, ...",male,true,bullet,field_partner
1419374,1067288,Ubale,English,Ubale is a Babban Gona maize farmer. He is a h...,Ubale is a Babban Gona maize farmer. He is a h...,75.0,75.0,funded,Farming,Agriculture,...,2016-05-13 05:37:01.000 +0000,19.0,2,1,1,NaN,male,true,bullet,field_partner
1419443,1068602,Shehu,English,Shehu is a Babban Gona maize farmer from Kargi...,Shehu is a Babban Gona maize farmer from Kargi...,175.0,350.0,expired,Farming,Agriculture,...,NaN,19.0,6,1,1,"#Parent, #Schooling, #Vegan",male,true,bullet,field_partner


In [15]:
# decidiamo di invertire le date del "planned expiration time" con quelle del "disburse time" nel caso analizzato in precedenza
loans['planned_expiration_time'],loans['disburse_time']=np.where(loans['disburse_time'] > loans['planned_expiration_time'],(loans['disburse_time'],loans['planned_expiration_time']),(loans['planned_expiration_time'],loans['disburse_time']))

In [16]:
#verifichiamo che lo scambio è stato effettuato correttamente
loans.loc[loans['disburse_time'] > loans['planned_expiration_time']]

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model


In [17]:
# utilizziamo datatime per leggere le date come tali e non come stringhe
loans['planned_expiration_time'] =  pd.to_datetime(loans['planned_expiration_time'], format='%Y-%m-%d %H:%M:%S.%f')
loans['disburse_time'] =  pd.to_datetime(loans['disburse_time'], format='%Y-%m-%d %H:%M:%S.%f')

In [18]:
loans['planned_expiration_time'].head()

0   2014-02-14 03:30:06+00:00
1   2014-03-26 22:25:07+00:00
2   2014-02-15 21:10:05+00:00
3   2014-02-21 03:10:02+00:00
4   2014-02-13 06:10:02+00:00
Name: planned_expiration_time, dtype: datetime64[ns, UTC]

In [19]:
loans['disburse_time'].head()

0   2013-12-22 08:00:00+00:00
1   2013-12-20 08:00:00+00:00
2   2014-01-09 08:00:00+00:00
3   2014-01-17 08:00:00+00:00
4   2013-12-17 08:00:00+00:00
Name: disburse_time, dtype: datetime64[ns, UTC]

In [20]:
# calcoliamo la duration creando una nuova colonna
loans['duration']=loans['planned_expiration_time']-loans['disburse_time']

In [21]:
loans['duration'].head()

0   53 days 19:30:06
1   96 days 14:25:07
2   37 days 13:10:05
3   34 days 19:10:02
4   57 days 22:10:02
Name: duration, dtype: timedelta64[ns]

In [22]:
missingTime=loans[["planned_expiration_time","disburse_time", "duration"]]
missingTime.head()

,planned_expiration_time,disburse_time,duration
0,2014-02-14 03:30:06+00:00,2013-12-22 08:00:00+00:00,53 days 19:30:06
1,2014-03-26 22:25:07+00:00,2013-12-20 08:00:00+00:00,96 days 14:25:07
2,2014-02-15 21:10:05+00:00,2014-01-09 08:00:00+00:00,37 days 13:10:05
3,2014-02-21 03:10:02+00:00,2014-01-17 08:00:00+00:00,34 days 19:10:02
4,2014-02-13 06:10:02+00:00,2013-12-17 08:00:00+00:00,57 days 22:10:02


In [23]:
#verifichiamo che se una delle due date è mancante anche la duration è mancante
missingTime[missingTime['duration'].isna()]

,planned_expiration_time,disburse_time,duration
64,NaT,2010-11-18 08:00:00+00:00,NaT
65,NaT,2010-11-25 08:00:00+00:00,NaT
66,NaT,2010-11-25 08:00:00+00:00,NaT
67,NaT,2010-11-16 08:00:00+00:00,NaT
69,NaT,2010-11-10 08:00:00+00:00,NaT
...,...,...,...
1419523,NaT,2005-04-14 05:27:55+00:00,NaT
1419524,2017-02-09 23:07:54+00:00,NaT,NaT
1419525,2015-09-21 08:06:19+00:00,NaT,NaT
1419538,2017-02-25 20:21:21+00:00,NaT,NaT


# Punto 3

Find the lenders that have funded at least twice.


In [24]:
# raggruppiamo per lenders e contiamo quante volte è presente ciascuno di essi
twice=fin_fin.groupby(['lenders']).lenders.agg('count').to_frame('count').reset_index()

In [25]:
#selezioniamo i lenders presenti almeno due volte
twice.loc[twice['count'] >= 2]

,lenders,count
0,000,40
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
...,...,...
1383792,zzinnia,28
1383793,zzivnk7094,4
1383794,zzmcfate,1730
1383796,zzrvmf8538,15


# Punto 4 

For each country, compute how many loans have involved that country as borrowers.


In [26]:
# raggruppiamo i loans per ogni Stato e vediamo quante volte è presente ciascuno di essi 
loans.groupby('country_code').size()

country_code
AF     2337
AL     3075
AM    13952
AZ    10172
BA      608
      ...  
XK     2178
YE     4206
ZA      633
ZM     1277
ZW     5513
Length: 95, dtype: int64

# Punto 5

For each country, compute the overall amount of money borrowed.


In [27]:
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,...,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,...,7.0,3,2,1,NaN,female,true,irregular,field_partner,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,...,8.0,11,2,1,NaN,female,true,monthly,field_partner,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,...,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,...,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,...,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,57 days 22:10:02


In [28]:
# dalla tabella loans abbiamo selezionato il country code e il loan amount
amount=loans[['country_code', 'loan_amount']]
amount

,country_code,loan_amount
0,PH,125.0
1,HN,400.0
2,PK,400.0
3,KG,625.0
4,PH,425.0
...,...,...
1419602,KE,400.0
1419603,PH,300.0
1419604,GH,2425.0
1419605,PH,100.0


In [29]:
#abbiamo raggruppato per country code e abbiamo sommato il loan amount
overall_cc=amount.groupby(['country_code']).sum()
overall_cc.head()

,loan_amount
country_code,
AF,1967950.0
AL,4307350.0
AM,22950475.0
AZ,14784625.0
BA,477250.0


# Punto 6

Like the previous point, but expressed as a percentage of the overall amount lent.


In [30]:
lenders=pd.read_csv('lenders.csv')
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [31]:
# abbiamo creato una nuova colonna con le percentuali del loan per ogni stato sul totale
overall_cc['percentage']=(overall_cc['loan_amount']/overall_cc['loan_amount'].sum())*100
overall_cc['percentage']

country_code
AF    0.166578
AL    0.364598
AM    1.942654
AZ    1.251452
BA    0.040397
        ...   
XK    0.260964
YE    0.291519
ZA    0.085198
ZM    0.167511
ZW    0.495335
Name: percentage, Length: 95, dtype: float64

# Punto 7

Like the three previous points, but split for each year (with respect to disburse time).


In [32]:
anno = pd.DatetimeIndex(loans['disburse_time'])

In [33]:
loans['year']=anno.year

In [34]:
#abbiamo raggruppato per country code e abbiamo visualizzato il numero di loan per ogni anno
loans.groupby(['country_code','year']).size()

country_code  year  
AF            2007.0     408
              2008.0     370
              2009.0     678
              2010.0     632
              2011.0     247
                        ... 
ZW            2013.0     426
              2014.0    2078
              2015.0     600
              2016.0     808
              2017.0    1079
Length: 740, dtype: int64

In [35]:
amount_x_year=loans[['loan_amount',  'country_code', 'year']]

In [36]:
#abbiamo raggruppato per country code e year e abbiamo sommato loan amount
amount_x_year.groupby(['country_code','year']).sum()

loan_amount
country_code year               
AF           2007.0     194975.0
             2008.0     365375.0
             2009.0     585125.0
             2010.0     563350.0
             2011.0     245125.0
...                          ...
ZW           2013.0     678525.0
             2014.0    1311575.0
             2015.0     723625.0
             2016.0     788600.0
             2017.0    1237600.0

[740 rows x 1 columns]

In [37]:
#abbiamo calcolato la percentuale di loan diviso per country code nei diversi anni
overall_year=amount_x_year.groupby(['country_code','year']).sum()
overall_year['percentage']=(overall_year['loan_amount']/overall_year['loan_amount'].sum())*100
overall_year['percentage']

country_code  year  
AF            2007.0    0.016658
              2008.0    0.031216
              2009.0    0.049991
              2010.0    0.048131
              2011.0    0.020943
                          ...   
ZW            2013.0    0.057971
              2014.0    0.112057
              2015.0    0.061824
              2016.0    0.067375
              2017.0    0.105736
Name: percentage, Length: 740, dtype: float64

# Punto 8 

For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


In [38]:
# selezioniamo da loans solo le colonne loan id e loan amount
id_sum=loans[['loan_id',  'loan_amount']]
id_sum.head()

,loan_id,loan_amount
0,657307,125.0
1,657259,400.0
2,658010,400.0
3,659347,625.0
4,656933,425.0


In [39]:
fin_fin.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


In [40]:
# uniamo la tabella id_sum con la tabella normalizzata dei loan_lenders grazie all'attributo loan_id
each=pd.merge(fin_fin, id_sum, left_on="loan_id", right_on="loan_id")
each.head()

,loan_id,lenders,loan_amount
0,483693,muc888,1225.0
1,483693,sam4326,1225.0
2,483693,camaran3922,1225.0
3,483693,lachheb1865,1225.0
4,483693,rebecca3499,1225.0


In [41]:
# visualizziamo quante volte si ripete il loan_id per ottenere il numero di lender per ciascun loan
n_loans=each.groupby('loan_id').size()
n_loans

loan_id
84         3
85         2
86         3
88         3
89         4
          ..
1444051    1
1444053    1
1444058    1
1444063    1
1444065    1
Length: 1387428, dtype: int64

In [42]:
#creiamo il dataframe con il loan_id e il numero di lenders calcolato in precedenza
number = { 'number': n_loans } 
  
number2 = pd.DataFrame(number) 
number2

,number
loan_id,
84,3
85,2
86,3
88,3
89,4
...,...
1444051,1
1444053,1
1444058,1


In [43]:
# aggiungiamo alla tabella each il numero di lenders per ogni loan
each_number=pd.merge(each, number2, left_on="loan_id", right_on="loan_id")
each_number.head()

,loan_id,lenders,loan_amount,number
0,483693,muc888,1225.0,40
1,483693,sam4326,1225.0,40
2,483693,camaran3922,1225.0,40
3,483693,lachheb1865,1225.0,40
4,483693,rebecca3499,1225.0,40


In [44]:
# Dividendo il loan amount con il numero di lenders per loan otteniamo le quote individuali per ciascun lenders
each_number['individual']=each_number['loan_amount']/each_number['number']
each_number.head()

,loan_id,lenders,loan_amount,number,individual
0,483693,muc888,1225.0,40,30.625
1,483693,sam4326,1225.0,40,30.625
2,483693,camaran3922,1225.0,40,30.625
3,483693,lachheb1865,1225.0,40,30.625
4,483693,rebecca3499,1225.0,40,30.625


In [45]:
#selezioniamo dalla tabella precedente gli attributi lenders e individual
sin_lender=each_number[['lenders', 'individual']]

In [46]:
#raggruppiamo i lender per ottenere il totale dei finanziamenti effettuati da ognuno
sum_lenders=sin_lender.groupby(['lenders']).sum().reset_index(drop=False)
sum_lenders

,lenders,individual
0,000,1764.285078
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117
...,...,...
1383794,zzmcfate,66113.226325
1383795,zzpaghetti9994,51.020408
1383796,zzrvmf8538,576.978086
1383797,zzzsai,267.667370


# Punto 9

For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


In [47]:
lenders=pd.read_csv('lenders.csv')

In [48]:
import numpy as np

In [49]:
lenders.head()

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0


In [50]:
# selezioniamo gli attributi permanent name e country code
lend=lenders[['permanent_name','country_code']].copy()
lend.head()

,permanent_name,country_code
0,qian3013,NaN
1,reena6733,NaN
2,mai5982,NaN
3,andrew86079135,NaN
4,nguyen6962,NaN


In [51]:
# notiamo che nella tabella lenders più di un milione di righe non contengono il country code
# abbiamo deciso di calcolare la distribuzione dei country code all'interno della tabella lenders e abbiamo riempito i dati mancanti seguendo la stessa distribuzione
lend[lend['country_code'].isna()]

,permanent_name,country_code
0,qian3013,NaN
1,reena6733,NaN
2,mai5982,NaN
3,andrew86079135,NaN
4,nguyen6962,NaN
...,...,...
2349167,todd5695,NaN
2349168,kate40761039,NaN
2349169,janet7309,NaN
2349170,pj4198,NaN


*Inizio del calcolo per l'imputazione*

In [52]:
# raggruppiamo per country code per sapere quante volte è presente ciascuno stato
frequency=lend.groupby(['country_code']).size()
frequency

country_code
AD      15
AE    1043
AF     228
AG       8
AI       4
      ... 
YE     195
YT       2
ZA    1051
ZM      65
ZW      54
Length: 234, dtype: int64

In [53]:
#abbiamo creato una tabella utilizzando i dati ottenuti in precedenza
c_f={"frequency":frequency}
country_freq=pd.DataFrame(c_f)

country_freq['country_code']=country_freq.index
country_freq=country_freq.reset_index(drop=True)

country_freq=country_freq[['country_code','frequency']]
country_freq

,country_code,frequency
0,AD,15
1,AE,1043
2,AF,228
3,AG,8
4,AI,4
...,...,...
229,YE,195
230,YT,2
231,ZA,1051
232,ZM,65


In [54]:
country_freq['percentage']=(country_freq['frequency']/country_freq['frequency'].sum())*100
country_freq

,country_code,frequency,percentage
0,AD,15,0.001684
1,AE,1043,0.117120
2,AF,228,0.025602
3,AG,8,0.000898
4,AI,4,0.000449
...,...,...,...
229,YE,195,0.021897
230,YT,2,0.000225
231,ZA,1051,0.118018
232,ZM,65,0.007299


In [55]:
#creiamo due array con i volori della percentuale e del country code per poterli utilizzare nella funzione successiva
percentage=country_freq['percentage'].to_numpy()
country_code=country_freq['country_code'].to_numpy()
percentage /= percentage.sum()

In [56]:
#riempiamo i dati mancanti come spiegato precedentemente
lend['country_code'] = lend['country_code'].fillna(pd.Series(np.random.choice(
                                                     country_code,p=country_freq.percentage, size=len(lend))))

*fine dell'imputazione*

In [58]:
#abbiamo calcolato nuovamente la distribuzione degli stati dopo il riempimento e notiamo che la nuova distribuzione
#è in linea con quella precedente
frequency=lend.groupby(['country_code']).size()
frequency
c_f={"frequency":frequency}
country_freq=pd.DataFrame(c_f)

country_freq['country_code']=country_freq.index
country_freq=country_freq.reset_index(drop=True)

country_freq=country_freq[['country_code','frequency']]
country_freq.head()
country_freq['percentage']=(country_freq['frequency']/country_freq['frequency'].sum())*100
country_freq

,country_code,frequency,percentage
0,AD,39,0.001660
1,AE,2781,0.118382
2,AF,627,0.026690
3,AG,19,0.000809
4,AI,13,0.000553
...,...,...,...
229,YE,516,0.021965
230,YT,7,0.000298
231,ZA,2791,0.118808
232,ZM,172,0.007322


In [59]:
sum_lenders.head()

,lenders,individual
0,000,1764.285078
1,00000,1380.693644
2,0002,2472.563566
3,00mike00,52.631579
4,0101craign0101,2623.565117


In [60]:
# uniamo le tabella sum lenders e lend
# ipotizziamo che il permanent name della tabella lenders corrisponda al loan id della tabella loans
inner_sum_lenders=pd.merge(sum_lenders, lend, left_on="lenders", right_on="permanent_name", how='inner')
inner_sum_lenders.head()

,lenders,individual,permanent_name,country_code
0,000,1764.285078,000,US
1,00000,1380.693644,00000,DE
2,0002,2472.563566,0002,US
3,00mike00,52.631579,00mike00,ID
4,0101craign0101,2623.565117,0101craign0101,US


In [61]:
# Raggruppando per country code e sommando abbiamo trovato il totale della somma prestata per ciascuno stato
money_lent=inner_sum_lenders.groupby(['country_code']).sum().reset_index(drop=False)
money_lent.rename(columns={'individual':'money_lent'}, inplace=True)
money_lent.head()

,country_code,money_lent
0,AD,6.346324e+03
1,AE,1.955863e+06
2,AF,1.572800e+05
3,AG,6.904606e+02
4,AI,5.630703e+02


In [62]:
#ipotizziamo che il country code nella tabella loans faccia riferimenti al country code di chi riceve il prestito
# Raggruppando per country code e sommando abbiamo trovato il totale della somma presa in prestito da ciascuno stato
money_borrowed=overall_cc.groupby(['country_code']).sum().reset_index(drop=False)
money_borrowed.rename(columns={'loan_amount':'money_borrowed'}, inplace=True)
money_borrowed.head()

,country_code,money_borrowed,percentage
0,AF,1967950.0,0.166578
1,AL,4307350.0,0.364598
2,AM,22950475.0,1.942654
3,AZ,14784625.0,1.251452
4,BA,477250.0,0.040397


In [63]:
#abbiamo unito le due tabelle per poter fare la differenza tra le due somme
money_both = money_lent.merge(money_borrowed,how='left', on='country_code').drop("percentage", axis=1)
money_both.head()

,country_code,money_lent,money_borrowed
0,AD,6.346324e+03,NaN
1,AE,1.955863e+06,NaN
2,AF,1.572800e+05,1967950.0
3,AG,6.904606e+02,NaN
4,AI,5.630703e+02,NaN


In [64]:
#dato che il numero di stati presenti nella tabella in cui abbiamo calcolato i money lent (lenders) è superiore
#al numero di stati presenti nella tabella dove abbiamo calcolato i money borrowed, quando uniamo le due tabelle
#gli stati assenti in money_borrowed compaiono come NaN, quindi decidiamo di sostituirli con 0 poichè supponiamo
#che questi stati non abbiano preso in prestito niente
money_both.money_borrowed = money_both.money_borrowed.fillna(0)
money_both.head()

,country_code,money_lent,money_borrowed
0,AD,6.346324e+03,0.0
1,AE,1.955863e+06,0.0
2,AF,1.572800e+05,1967950.0
3,AG,6.904606e+02,0.0
4,AI,5.630703e+02,0.0


In [65]:
money_both['delta'] = money_both['money_lent'] - money_both['money_borrowed']

In [66]:
money_both.head()

,country_code,money_lent,money_borrowed,delta
0,AD,6.346324e+03,0.0,6.346324e+03
1,AE,1.955863e+06,0.0,1.955863e+06
2,AF,1.572800e+05,1967950.0,-1.810670e+06
3,AG,6.904606e+02,0.0,6.904606e+02
4,AI,5.630703e+02,0.0,5.630703e+02


# Punto 10

Which country has the highest ratio between the difference computed at the previous point and the population?


In [6]:
c_stat=pd.read_csv('country_stats.csv')

In [7]:
#selezioniamo gli attributi country code e population
c_stat2=c_stat[['country_code', 'population']]
c_stat2.head()

,country_code,population
0,IN,1339180127
1,NG,190886311
2,MX,129163276
3,PK,197015955
4,BD,164669751


In [69]:
# uniamo la tabella contenetente i soldi prestati e presi in prestito con le statistiche per paese
final_stat=pd.merge(money_both, c_stat2, left_on="country_code", right_on="country_code")
final_stat.head()

,country_code,money_lent,money_borrowed,delta,population
0,AE,1.955863e+06,0.0,1.955863e+06,9400145
1,AF,1.572800e+05,1967950.0,-1.810670e+06,35530081
2,AL,2.866083e+04,4307350.0,-4.278689e+06,2930187
3,AM,3.638544e+04,22950475.0,-2.291409e+07,2930450
4,AO,6.157827e+04,0.0,6.157827e+04,29784193


In [70]:
#calcoliamo per ogni paese il rapporto tra la differenza precedentemente calcolata e la population
final_stat['ratio']=final_stat['delta']/final_stat['population']
final_stat.head()

,country_code,money_lent,money_borrowed,delta,population,ratio
0,AE,1.955863e+06,0.0,1.955863e+06,9400145,0.208067
1,AF,1.572800e+05,1967950.0,-1.810670e+06,35530081,-0.050962
2,AL,2.866083e+04,4307350.0,-4.278689e+06,2930187,-1.460210
3,AM,3.638544e+04,22950475.0,-2.291409e+07,2930450,-7.819307
4,AO,6.157827e+04,0.0,6.157827e+04,29784193,0.002067


In [71]:
#selezioniamo il paese con il valore del rapporto più alto
final_stat[final_stat['ratio']==final_stat['ratio'].max()]

,country_code,money_lent,money_borrowed,delta,population,ratio
114,NO,2.105485e+07,0.0,2.105485e+07,5305383,3.968582


# Punto 11

Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?


In [71]:
final_stat.head()

,country_code,money_lent,money_borrowed,percentage,delta,population,ratio
0,AE,1.972226e+06,0.0,NaN,1.972226e+06,9400145,0.209808
1,AF,1.769565e+05,1967950.0,0.166578,-1.790993e+06,35530081,-0.050408
2,AL,1.955838e+04,4307350.0,0.364598,-4.287792e+06,2930187,-1.463317
3,AM,4.209402e+04,22950475.0,1.942654,-2.290838e+07,2930450,-7.817359
4,AO,4.252204e+04,0.0,NaN,4.252204e+04,29784193,0.001428


In [72]:
pop_below=c_stat[['country_code','population_below_poverty_line']].copy()
pop_below.head()

,country_code,population_below_poverty_line
0,IN,21.9
1,NG,70.0
2,MX,46.2
3,PK,29.5
4,BD,31.5


In [73]:
#aggiungiamo la tabella del punto 11 la percentuale della popolazione sotto la soglia della povertà
pop_merged=pd.merge(final_stat, pop_below, left_on="country_code", right_on="country_code")
del pop_merged['ratio']
pop_merged.head()

,country_code,money_lent,money_borrowed,percentage,delta,population,population_below_poverty_line
0,AE,1.972226e+06,0.0,NaN,1.972226e+06,9400145,19.5
1,AF,1.769565e+05,1967950.0,0.166578,-1.790993e+06,35530081,35.8
2,AL,1.955838e+04,4307350.0,0.364598,-4.287792e+06,2930187,14.3
3,AM,4.209402e+04,22950475.0,1.942654,-2.290838e+07,2930450,32.0
4,AO,4.252204e+04,0.0,NaN,4.252204e+04,29784193,40.5


In [74]:
#calcoliamo per ogni paese il numero di persone sopra la soglia della povertà
pop_merged['pop_upper']=pop_merged['population']*(1-(pop_merged['population_below_poverty_line']/100))
pop_merged.head()

,country_code,money_lent,money_borrowed,percentage,delta,population,population_below_poverty_line,pop_upper
0,AE,1.972226e+06,0.0,NaN,1.972226e+06,9400145,19.5,7.567117e+06
1,AF,1.769565e+05,1967950.0,0.166578,-1.790993e+06,35530081,35.8,2.281031e+07
2,AL,1.955838e+04,4307350.0,0.364598,-4.287792e+06,2930187,14.3,2.511170e+06
3,AM,4.209402e+04,22950475.0,1.942654,-2.290838e+07,2930450,32.0,1.992706e+06
4,AO,4.252204e+04,0.0,NaN,4.252204e+04,29784193,40.5,1.772159e+07


In [75]:
#calcoliamo per ogni country code il rapporto tra la differenza tra i soldi prestati e presi in prestito e la popolazione sopra la soglia della povertà 
pop_merged['up_ratio']=pop_merged['delta']/pop_merged['pop_upper']
pop_merged.head()

,country_code,money_lent,money_borrowed,percentage,delta,population,population_below_poverty_line,pop_upper,up_ratio
0,AE,1.972226e+06,0.0,NaN,1.972226e+06,9400145,19.5,7.567117e+06,0.260631
1,AF,1.769565e+05,1967950.0,0.166578,-1.790993e+06,35530081,35.8,2.281031e+07,-0.078517
2,AL,1.955838e+04,4307350.0,0.364598,-4.287792e+06,2930187,14.3,2.511170e+06,-1.707487
3,AM,4.209402e+04,22950475.0,1.942654,-2.290838e+07,2930450,32.0,1.992706e+06,-11.496117
4,AO,4.252204e+04,0.0,NaN,4.252204e+04,29784193,40.5,1.772159e+07,0.002399


In [76]:
#selezioniamo il country code con il rapporto più elevato
pop_merged[pop_merged['up_ratio']==pop_merged['up_ratio'].max()]

,country_code,money_lent,money_borrowed,percentage,delta,population,population_below_poverty_line,pop_upper,up_ratio
24,CA,9.406175e+07,50000.0,0.004232,9.401175e+07,36624199,9.4,3.318152e+07,2.833256
